In [0]:
%pip install openai pandas openpyxl

In [0]:
import os
import openai
import pandas as pd
import json
import openpyxl
import random

In [0]:

from openai import OpenAI
import os

# How to get your Databricks token: https://docs.databricks.com/en/dev-tools/auth/pat.html
DATABRICKS_TOKEN = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

client = OpenAI(
    api_key=DATABRICKS_TOKEN,
    base_url="https://adb-2220072380334347.7.azuredatabricks.net/serving-endpoints"
)

response = client.chat.completions.create(
    model="databricks-meta-llama-3-3-70b-instruct",
    messages=[
        {
            "role": "user",
            "content": "You are a helpful assistant whose role is to quality assure the content in the input data. Your tasks inclValidating accuracy of the provided information.Cross-checking consistency of the content with any tables, plots, or visual data representations given.Highlighting discrepancies or errors between the written content and the numerical/graphical data."
        }
    ],
    max_tokens=5000
)

print(response.choices[0].message.content)

In [0]:
from openai import OpenAI
import os

# How to get your Databricks token: https://docs.databricks.com/en/dev-tools/auth/pat.html
DATABRICKS_TOKEN = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

client = OpenAI(
    api_key=DATABRICKS_TOKEN,
    base_url="https://adb-2220072380334347.7.azuredatabricks.net/serving-endpoints"
)

response = client.chat.completions.create(
    model="databricks-claude-3-7-sonnet",
    messages=[
        {
            "role": "user",
            "content": "You are a helpful assistant whose role is to quality assure the content in the input data. Your tasks inclValidating accuracy of the provided information.Cross-checking consistency of the content with any tables, plots, or visual data representations given.Highlighting discrepancies or errors between the written content and the numerical/graphical data."
        }
    ],
    max_tokens=5000
)

print(response.choices[0].message.content)

In [0]:
%pip install python-docx
import pandas as pd
import docx
import re

In [0]:
import docx

# Load the Word file which is in the same folder
doc = docx.Document("KS4_Commentary_202324_revised.docx")

# Display the content of the Word file
for para in doc.paragraphs:
    print(para.text)

In [0]:
# Load the Word file which is in the same folder
import docx
# Load the Word file which is in the same folder
doc = docx.Document("ecf_text.docx")


# Display the content of the Word file
for para in doc.paragraphs:
    print(para.text)

In [0]:
import pandas as pd

# Load the CSV file
csv_df = pd.read_csv("teacher_leader_development_ecf_2024_25.csv")
meta_df =pd.read_csv("teacher_leader_development_ecf_2024_25.meta.csv")
mentors =pd.read_csv("teacher_leader_development_ecf_mentor_2024_25.csv")
mentors_meta = pd.read_csv("teacher_leader_development_ecf_mentor_2024_25.meta.csv")
retention = pd.read_csv("teacher_leader_development_ecf_retention_2024_25.csv")
retention_meta = pd.read_csv("teacher_leader_development_ecf_retention_2024_25.meta.csv")

In [0]:
##checking for headlines: 
text = "\n".join([para.text for para in doc.paragraphs if para.text.strip()])

# Initialize Databricks LLM client
DATABRICKS_TOKEN = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

client = OpenAI(
    api_key=DATABRICKS_TOKEN,
    base_url="https://adb-2220072380334347.7.azuredatabricks.net/serving-endpoints"   # replace with your Databricks endpoint
)

# Simple query to check if the LLM understands
response = client.chat.completions.create(
    model="databricks-claude-3-7-sonnet",
    messages=[
        {
            "role": "user",
            "content": f"Here is my document text:\n\n{text}\n\nCan you list any headlines you detect?"
        }
    ],
    max_tokens=500
)

doc_headlines = response.choices[0].message.content.split("\n")
doc_headlines = [h.strip() for h in doc_headlines if h.strip()]

In [0]:
print(doc_headlines)

In [0]:
##### now onto the csv files: only keeping the values which are relavnt for healdines:  
# #Function to filter a dataframe: keep 'time_period' plus columns from 'breakdown' to 'group_name'
def filter_df(df):
    if "breakdown" in df.columns and "time_period" in df.columns:
        start_idx = df.columns.get_loc("breakdown")
        # Keep all columns from "breakdown" to the end
        cols_to_keep = ["time_period"] + list(df.columns[start_idx:])
        return df[cols_to_keep]
    else:
        print("One or more required columns are missing.")
        return df


# Apply filtering
filtered_csv_df = filter_df(csv_df)
filtered_mentors = filter_df(mentors)
filtered_retention = filter_df(retention)





In [0]:
# Preview
print(filtered_csv_df.head())
print(filtered_mentors.head())
print(filtered_retention.head())

In [0]:
import re

def extract_numbers_from_df(df):
    """Extract all integers from all columns in a DataFrame."""
    numbers = set()
    for col in df.columns:
        # Convert everything to string to make regex work
        for val in df[col].astype(str):
            found = re.findall(r'\d+', val)
            numbers.update(map(int, found))
    return numbers


In [0]:
# Extract numbers from each filtered DataFrame
csv_numbers = extract_numbers_from_df(filtered_csv_df)
mentor_numbers = extract_numbers_from_df(filtered_mentors)
retention_numbers = extract_numbers_from_df(filtered_retention)

In [0]:
# Combine all numbers
all_csv_numbers = csv_numbers | mentor_numbers | retention_numbers



In [0]:
print(doc_headlines)

In [0]:
#### extract numbers from text data
import re

# Join all lines into one string
text = " ".join(doc_headlines)

# Regex to find numbers (integers, decimals, and numbers with commas)
all_numbers = re.findall(r'\d[\d,]*\.?\d*', text)

# Convert numbers to int or float
numbers = []
for num in all_numbers:
    # Ignore years like 2021-2024
    if re.match(r'^(20\d{2})$', num):
        continue
    # Ignore fiscal-year style numbers like 2023/24
    if re.match(r'^20\d{2}/\d{2}$', num):
        continue
    # Convert to int or float
    numbers.append(float(num.replace(',', '')) if '.' in num else int(num.replace(',', '')))

print(numbers)




In [0]:
# Combine all CSV numbers into one list
all_csv_numbers = csv_numbers | mentor_numbers | retention_numbers

In [0]:

# Option 2: Check if all numbers from text exist in CSVs
all_in_csv = all(num in all_csv_numbers for num in numbers)
print("All numbers from text exist in CSVs?", all_in_csv)

In [0]:
diff_text_not_in_csv = [num for num in numbers if num not in all_csv_numbers]
diff_csv_not_in_text = [num for num in all_csv_numbers if num not in numbers]

print("Numbers in text but not in CSVs:", diff_text_not_in_csv)


In [0]:
print(doc_headlines)

In [0]:



sections = {}
current_heading = None
current_content = []

def is_bold_heading(paragraph):
    """Check if the paragraph contains bold text (used as heading)."""
    return any(run.bold for run in paragraph.runs) and paragraph.text.strip()

for para in doc.paragraphs:
    text = para.text.strip()
    if not text:
        continue

    if is_bold_heading(para):
        if current_heading and current_content:
            sections[current_heading] = "\n".join(current_content)
            current_content = []
        current_heading = text
    else:
        current_content.append(text)

if current_heading and current_content:
    sections[current_heading] = "\n".join(current_content)




In [0]:
# Convert into DataFrames
dfs = {heading: pd.DataFrame({"Content": content.split("\n")}) 
       for heading, content in sections.items()}

# Initialize Databricks LLM client
DATABRICKS_TOKEN = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

client = OpenAI(
    api_key=DATABRICKS_TOKEN,
    base_url="https://adb-2220072380334347.7.azuredatabricks.net/serving-endpoints"
)

def extract_headlines_from_section(df, section_name):
    """Send one section to the LLM and return detected headlines."""
    section_text = "\n".join(df["Content"].tolist())
    
    response = client.chat.completions.create(
        model="databricks-claude-3-7-sonnet",
        messages=[
            {
                "role": "user",
                "content": f"Here is my document section: {section_name}\n\n{section_text}\n\nCan you list any headlines you detect?"
            }
        ],
        max_tokens=500
    )

    raw_output = response.choices[0].message.content
    doc_headlines = [h.strip() for h in raw_output.split("\n") if h.strip()]
    return doc_headlines


In [0]:
print(doc_headlines)

In [0]:
# Loop through each section and extract headlines
all_headlines = {}
for heading, df in dfs.items():
    print(f"\n### DataFrame for: {heading} ###")
    print(df)

    section_headlines = extract_headlines_from_section(df, heading)
    all_headlines[heading] = section_headlines

    print("\nDetected headlines:", section_headlines)

In [0]:
############# try ######
import docx

doc = docx.Document("ecf_text.docx")

sections = {}
current_heading = None
current_content = []

def is_bold_heading(paragraph):
    """Check if the entire paragraph is bold (heading)."""
    return any(run.bold for run in paragraph.runs) and paragraph.text.strip()

for para in doc.paragraphs:
    text = para.text.strip()
    if not text:
        continue

    if is_bold_heading(para):
        # Save previous section if it exists
        if current_heading and current_content:
            sections[current_heading] = "\n".join(current_content)
            current_content = []
        current_heading = text  # new heading
    else:
        current_content.append(text)

# Save last section
if current_heading and current_content:
    sections[current_heading] = "\n".join(current_content)

# Display results
for heading, content in sections.items():
    print(f"=== {heading} ===")
    print(content)
    print()


    sections = {}
current_heading = None
current_content = []

def is_bold_heading(paragraph):
    """Check if the paragraph contains bold text (used as heading)."""
    return any(run.bold for run in paragraph.runs) and paragraph.text.strip()

for para in doc.paragraphs:
    text = para.text.strip()
    if not text:
        continue

    if is_bold_heading(para):
        # Save previous section if it exists
        if current_heading and current_content:
            sections[current_heading] = "\n".join(current_content)
            current_content = []
        current_heading = text
    else:
        current_content.append(text)

# Save last section
if current_heading and current_content:
    sections[current_heading] = "\n".join(current_content)

# Convert each section into its own DataFrame
dfs = {}
for heading, content in sections.items():
    dfs[heading] = pd.DataFrame({"Content": content.split("\n")})

# Example: access a specific DataFrame by heading
for heading, df in dfs.items():
    print(f"\n### DataFrame for: {heading} ###")
    print(df)

In [0]:
response = client.chat.completions.create(
    model="databricks-claude-3-7-sonnet",
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant that checks summaries for accuracy against provided data."
        },
        {
            "role": "user",
            "content": f"""
Here is the source text:
{full_text}

Here is the table data:
{table_data}

Here is a summary I wrote:
{user_summary}

Check if the summary accurately reflects the text AND the table. 
- Highlight any inconsistencies. 
- Suggest corrections if needed. 
- If everything is correct, just say 'Summary is consistent with data'.
"""
        }
    ],
    max_tokens=5000
)

print(response.choices[0].message.content)

In [0]:
# Example: doc_headlines is a list of summaries/headlines
# csv_numbers is a dict or list of numbers extracted from your CSV

for headline in doc_headlines:
    response = client.chat.completions.create(
        model="databricks-claude-3-7-sonnet",
        messages=[
            {
                "role": "system",
                "content": "You are an AI assistant that checks summaries for factual accuracy against numeric data and corrects any mistakes."
            },
            {
                "role": "user",
                "content": f"""
Check the following summary for accuracy:

Summary:
{headline}

Reference numbers from the table:
{csv_numbers}

- Identify any factual inaccuracies in the summary compared to the table numbers.
- Suggest a corrected version if needed.
- If the summary is correct, just confirm it.
"""
            }
        ],
        max_tokens=1000
    )

    print("Original headline:", headline)
    print("LLM feedback:", response.choices[0].message.content)
    print("-" * 50)
